In [11]:
# @ written by shubham bansal:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
#from geopy.distance import great_circle
#import pygmaps
from scipy.spatial import ConvexHull
from datetime import datetime


%matplotlib inline

print('hello')


hello


In [12]:
def Data_prepration(Data_frame, newfilename):
                ### function defination 
                # data Given 
                #data_of_buses=pd.read_csv('/home/b/Desktop/Gis_project/Gis_project/data/356_EW/1july_356ew/150218661.csv')
                data_of_buses=Data_frame.copy()
                #print(data_of_buses.head())
                # data extraction from data frame
                #data_of_buses['Date'], data_of_buses['Time'] = data_of_buses['IST_DATE'].str.split(' ', 1).str
                data_of_buses=data_of_buses.rename(index=str,columns={"IST_TIME": "Time"})
              #  print(data_of_buses.head())
                Cleaned_data=data_of_buses[['LAT','LONGITUDE','Time','Acc_distance']].copy()
                Cleaned_data=Cleaned_data.sort_values('Time')
                #print(data_of_buses.head())
                #type(Cleaned_data)
                #print(Cleaned_data.head())
                Cleaned_data_array=Cleaned_data.as_matrix()
                Cleaned_data_array
                size=len(Cleaned_data_array)
                size
                Data_Train=pd.DataFrame(columns=['Start_Lat','Start_Longitude','Start_Time','End_Lat','End_Longitude','End_Time','Distance','Flag','Day_time'])
                Data_Train

                #time_x=00:00:00
                S_lat=Cleaned_data_array[0][0]
                S_long=Cleaned_data_array[0][1]
                S_time=Cleaned_data_array[0][2]
                
                E_lat=Cleaned_data_array[0][0]
                E_long=Cleaned_data_array[0][1]
                E_time=Cleaned_data_array[0][2]
                
                Acc_start=Cleaned_data_array[0][3]
                Acc_end=Cleaned_data_array[0][3]
                FMT = '%H:%M:%S'
                time_diff=0
                flag=0
                for i in range(1,size):
                       # time1=(Cleaned_data_array[i][2])
                        time2=(Cleaned_data_array[i][2])
                        tdelta = abs(datetime.strptime(S_time, FMT) - datetime.strptime(time2, FMT))
                        #print(tdelta)
                        time_diff=tdelta.total_seconds()
                       # print(time_diff)
                       
                        if time_diff>600 :
                            Acc_end=Cleaned_data_array[i-1][3]
                            E_time= Cleaned_data_array[i-1][2]
                            E_lat=Cleaned_data_array[i-1][0]
                            E_long=Cleaned_data_array[i-1][1]
                            diff=Acc_end-Acc_start  
                            #Actual_time_difference=E_time-S_time
                            Actual_time_difference= datetime.strptime(E_time, FMT)-datetime.strptime(S_time, FMT) 
                         #   print(S_time,E_time)
                            #print(Actual_time_difference)
                            Actual_time_difference_total= Actual_time_difference.total_seconds()
                           # print(Actual_time_difference_total)
                            #print(" helllo")
                            threshold_distance=(Actual_time_difference_total*(2.78))
                            if  diff!=0:
                                    if diff<threshold_distance :
                                        leng=len(Data_Train)
                                        if leng==0 :
                                             # Day_time=(datetime.strptime(S_time, FMT)-datetime.strptime(time_x, FMT))
                                             # Day_time_total=Day_time.total_seconds()
                                              s=S_time
                                              Total_day_time=sum(int(i) * 60**index for index, i in enumerate(s.split(":")[::-1])) 
                                            #  print(Total_day_time)
                                              Data_Train=Data_Train.append({'Start_Lat':S_lat,'Start_Longitude':S_long,'Start_Time':S_time,'End_Lat':E_lat,'End_Longitude':E_long,'End_Time':E_time,'Distance':diff,'Flag':flag,'Day_time':Total_day_time},ignore_index=True)
                                        else :
                                           # print (leng)
                                            #lat=(int)((Data_Train.iloc[leng-1][3])*1000000)
                                          #  print (lat)
                                           #long=(int)((Data_Train.iloc[leng-1][4])*1000000)
                                            #if lat==(int)(S_lat*1000000) and long ==(int)(S_long*1000000) :
                                           # if ((Data_Train.iloc[leng-1][5])==S_time)
                                            if(True):
                                                current_time=S_time
                                                prvious_time=Data_Train.iloc[leng-1][5]
                                                #print(current_time,prvious_time)
                                                #Day_time=(datetime.strptime(Start_time, FMT)).total_seconds()
                                                
                                              #  Day_time=(datetime.strptime(S_time, FMT))
                                               # Day_time_total=Day_time.total_seconds()
                                                time_differ = (abs(datetime.strptime(current_time, FMT) - datetime.strptime(prvious_time, FMT))).total_seconds()
                                                #(Data_Train.iloc[leng-1][5])
                                                #print(current_time,prvious_time,time_differ)
                                                if(time_differ<30):
                                                    flag=flag+1
                                                else:
                                                    flag=0
                                            s=S_time
                                            Total_day_time=sum(int(i) * 60**index for index, i in enumerate(s.split(":")[::-1])) 
                                           # print(Total_day_time)
                                            Data_Train=Data_Train.append({'Start_Lat':S_lat,'Start_Longitude':S_long,'Start_Time':S_time,'End_Lat':E_lat,'End_Longitude':E_long,'End_Time':E_time,'Distance':diff,'Flag':flag,'Day_time':Total_day_time},ignore_index=True)

                            S_lat=Cleaned_data_array[i][0]
                            S_long=Cleaned_data_array[i][1]
                            S_time=Cleaned_data_array[i][2]
                            Acc_start=Cleaned_data_array[i-1][3]
                Data_Train.to_csv(newfilename,index=False)
                return Data_Train

            

In [13]:
def Load_file(filename, newfilename):
    Data_Train=pd.read_csv(filename)
    Data_Train_final=Data_prepration(Data_Train, newfilename)

In [14]:
import os
root = '/home/b/Desktop/Traffic_prediction/gis_project/Data_route_wise/356_EW'
for subdir, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith('-checkpoint.csv'):
            continue
#         print(subdir, filename)
        folders = subdir.split(sep='/')
        newfilename = folders[7]+'_'+folders[8]+'_'+filename
      #  print(os.path.join(subdir,filename),newfilename)
        print(newfilename)
        Load_file(os.path.join(subdir,filename), newfilename)

356_EW_4july_150218661.csv
356_EW_4july_150222706.csv
356_EW_4july_150220682.csv
356_EW_4july_150220615.csv
356_EW_5july_150218661.csv
356_EW_5july_150222706.csv
356_EW_5july_150220682.csv
356_EW_5july_150220615.csv
356_EW_1july_150218661.csv
356_EW_1july_150222706.csv
356_EW_1july_150220682.csv
356_EW_1july_150220615.csv


In [5]:
#     Data_Train=pd.read_csv('/home/b/Desktop/Gis_project/Gis_project/data/356_EW/5july_356ew/150222706.csv')
#     Data_Train_final=Data_prepration(Data_Train)

In [6]:
# def Data_prepration(Data_frame):
#                 ### function defination 
#                 # data Given 
#                 #data_of_buses=pd.read_csv('/home/b/Desktop/Gis_project/Gis_project/data/356_EW/1july_356ew/150218661.csv')
#                 data_of_buses=Data_frame.copy()
#                 #print(data_of_buses.head())
#                 # data extraction from data frame
#                 #data_of_buses['Date'], data_of_buses['Time'] = data_of_buses['IST_DATE'].str.split(' ', 1).str
#                 data_of_buses=data_of_buses.rename(index=str,columns={"IST_TIME": "Time"})
#               #  print(data_of_buses.head())
#                 Cleaned_data=data_of_buses[['LAT','LONGITUDE','Time','Acc_distance']].copy()
#                 #type(Cleaned_data)
#                 Cleaned_data.head()
#                 Cleaned_data_array=Cleaned_data.as_matrix()
#                 Cleaned_data_array
#                 size=len(Cleaned_data_array)
#                 size
#                 Data_Train=pd.DataFrame(columns=['Start_Lat','Start_Longitude','Start_Time','End_Lat','End_Longitude','End_Time','Distance','Flag'])
#                 Data_Train


#                 S_lat=Cleaned_data_array[0][0]
#                 S_long=Cleaned_data_array[0][1]
#                 S_time=Cleaned_data_array[0][2]
#                 E_lat=Cleaned_data_array[0][0]
#                 E_long=Cleaned_data_array[0][1]
#                 E_time=Cleaned_data_array[0][2]
#                 Acc_start=Cleaned_data_array[0][3]
#                 Acc_end=Cleaned_data_array[0][3]
#                 FMT = '%H:%M:%S'
#                 time_diff=0
#                 flag=1
#                 for i in range(1,size):
#                        # time1=(Cleaned_data_array[i][2])
#                         time2=(Cleaned_data_array[i][2])
#                         tdelta = abs(datetime.strptime(S_time, FMT) - datetime.strptime(time2, FMT))
#                         time_diff=tdelta.total_seconds()
#                         if time_diff>600 :
#                             Acc_end=Cleaned_data_array[i-1][3]
#                             E_time= Cleaned_data_array[i-1][2]
#                             E_lat=Cleaned_data_array[i-1][0]
#                             E_long=Cleaned_data_array[i-1][1]
#                             diff=Acc_end-Acc_start
#                             if diff==0 :
#                                 S_lat=Cleaned_data_array[i-1][0]
#                                 S_long=Cleaned_data_array[i-1][1]
#                                 S_time=Cleaned_data_array[i-1][2]
#                                 Acc_start=Cleaned_data_array[i-1][3]
#                             else :
#                                     if diff<1667:
#                                         leng=len(Data_Train)
#                                         if leng==0 :
#                                               Data_Train=Data_Train.append({'Start_Lat':S_lat,'Start_Longitude':S_long,'Start_Time':S_time,'End_Lat':E_lat,'End_Longitude':E_long,'End_Time':E_time,'Distance':diff,'Flag':flag},ignore_index=True)
#                                         else :
#                                            # print (leng)
#                                             lat=(int)((Data_Train.iloc[leng-1][3])*1000000)
#                                           #  print (lat)
#                                             long=(int)((Data_Train.iloc[leng-1][4])*1000000)
#                                             if lat==(int)(S_lat*1000000) and long ==(int)(S_long*1000000) :
#                                                 flag=flag+1
#                                             else:
#                                                 flag=1
#                                             Data_Train=Data_Train.append({'Start_Lat':S_lat,'Start_Longitude':S_long,'Start_Time':S_time,'End_Lat':E_lat,'End_Longitude':E_long,'End_Time':E_time,'Distance':diff,'Flag':flag},ignore_index=True)



#                                     S_lat=Cleaned_data_array[i-1][0]
#                                     S_long=Cleaned_data_array[i-1][1]
#                                     S_time=Cleaned_data_array[i-1][2]
#                                     Acc_start=Cleaned_data_array[i-1][3]
#                 Data_Train.to_csv("Data_train_buses_5_150222706.csv",index=False)
#                 return Data_Train

            